In [1]:

from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import normalize
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from boruta import BorutaPy 

    

In [2]:
def sensitivity_score(ypred,y):
    tp=np.sum((ypred==1) & (y==1))
   # print("tp")
    totmal=np.sum(y==1)
   # print(tp)
    #print("sum ypred:")
    #print(sum(ypred==1))
    #print("sum y:")
    #print(sum(y==1))
    #print("totmal")
    
    #Non mi ritrovo nel dividere tutto per "ypred.shape[0]" la sensibilità è TruePos/totmalati
    
    #print(totpos)
    print(tp/totmal)
    
    

In [3]:
#funzione per calcolare F_score (o almeno dovrebbe) TruePos/(TruePos+ 1/2 (FalsePos+FalseNeg)) 
def f_score(ypred, y):
    tp=np.sum((ypred==1) & (y==1))
    fp=np.sum(ypred==1)-tp
    tn=np.sum((ypred==0) & (y==0))
    fn=np.sum(ypred==0)-tn
    
    print(tp/(tp+0.5*(fp+fn)))

In [4]:

#Funzione per calcolare la specificità (o almeno dovrebbe) TrueNeg/TotSani

def specificity_score(ypred,y):
    tn=np.sum((ypred==0) & (y==0))
    totsani=np.sum(y==0)
    
    print (tn/totsani)

In [5]:
#carico il dataset
data = pd.read_csv(r'C:\Users\Utente\anaconda3\Lib\site-packages\pandas\io\data_covnet_score-imputed_missRF_increasing_1.txt')
print(data.shape)
nit=15

#il comando sotto non mi serve: Converte i dati categorici in dati classificabili.
#data = pd.get_dummies(data, drop_first=True, dummy_na=True)
#print(data.shape)

y=data['LABEL']
y=y.to_numpy()

parzial_features = list()

altri=data
altri=altri.to_numpy()


features = [f for f in data.columns if f not in ['LABEL']]
#print(len(features))

X = data[features].values

#ravel() mi fa diventare l'array monodimensionale.
Y = data['LABEL'].values.ravel()

rf = RandomForestClassifier(n_estimators = 100, max_depth=11, max_features=None, max_samples=0.75,class_weight="balanced")


o=0

#creo un array monodimensionale lungo quanto "altri" ma solo di 0. 
y_pred=y-y
n_fold=10

#crea un oggetto pronto ad operare: quando gli arriva in input qualcosa lo divide in 10 pezzettini con la stessa proporzione
skf = StratifiedKFold(n_splits=n_fold)


for niter in range(nit):

    for train_index, test_index in skf.split(X, Y):
            boruta_feature_selector = BorutaPy(rf, n_estimators='auto', verbose=0, max_iter = 50, perc = 40)

            X_train, X_test = X[train_index,:], X[test_index,:]
            #usa boruta per selezionare le features più disciriminative dal training set
            #prendi solo le features più significative sia nel training set che nel test set
            y_train, y_test = Y[train_index], Y[test_index]
        
            boruta_feature_selector.fit(X_train, y_train)
            indexes = np.where(boruta_feature_selector.support_ == True)
            
            
            X_train_selected=boruta_feature_selector.transform(X_train)
            X_test_selected=boruta_feature_selector.transform(X_test)
            o=o+1
            print(o)
            #print(head(X_train_selected))
            clf = rf.fit(X_train_selected, y_train)
           
            y_pred[test_index] =clf.predict(X_test_selected)

           # array = boruta_feature_selector.transform(X)

          
            for x in np.nditer(indexes):
                parzial_features.append(features[x])

        #fine ciclo for
        
    #calcolare media di sensitivity ecc. QUI
  
    
X_filtered = boruta_feature_selector.transform(X)
final_features = list()
indexes = np.where(boruta_feature_selector.support_ == True)
#for x in np.nditer(indexes):
 #   final_features.append(features[x])
#print(final_features)

    

    
    

(301, 42)
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


In [9]:
print()
for i in features:
  c=parzial_features.count(i)
  print(i,'        ', c, c/(nit*n_fold))
  if (c>90):
    final_features.append(i)
print(final_features)


CAT.Fever          3 0.02
CAT.Cough          0 0.0
CAT.Dyspnea          8 0.05333333333333334
CAT.IR          0 0.0
CAT.Myalgias          0 0.0
CAT.Other          0 0.0
CAT.Syncope          0 0.0
CAT.Asthenia          0 0.0
CAT.Vomiting.Nausea          98 0.6533333333333333
CAT.Diarrhea          13 0.08666666666666667
CAT.Headache          0 0.0
CAT.Pharingeal.pain          0 0.0
INT.No.Symptoms          137 0.9133333333333333
CAT.Pneumo.asthma          0 0.0
CAT.Pneumo.BPCO          0 0.0
CAT.Neoplasia.last.5.years          66 0.44
CAT.Smoke          0 0.0
CAT.Arterial.hypertension          0 0.0
CAT.Cardiovascular.pathologies          20 0.13333333333333333
CAT.Diabetes          76 0.5066666666666667
CAT.Obesity          0 0.0
CAT.Celebral.stroke          0 0.0
INT.No.Comorbidities          144 0.96
CAT.Sex          0 0.0
INT.Age          150 1.0
INT.Symptoms.No.days          149 0.9933333333333333
INT.usa.radio.score.MAX          92 0.6133333333333333
INT.radio.SCORE          149 0